### Dataset Creation

In [22]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)

# Define real-world part names
real_part_names = [
    "Hydraulic Pump", "Pressure Sensor", "Check Valve", "Ball Bearing", "Seal Ring",
    "Drive Shaft", "Cooling Fan", "Circuit Breaker", "Solenoid Valve", "Oil Filter",
    "Timing Belt", "Throttle Body", "Radiator Cap", "Ignition Coil", "Spark Plug",
    "Brake Disc", "Fuel Injector", "Air Compressor", "Water Pump", "Control Arm",
    "Battery Pack", "Alternator", "Starter Motor", "Clutch Plate", "Shock Absorber",
    "Turbocharger", "Oxygen Sensor", "Wiper Blade", "Axle Shaft", "Thermostat"
]

categories = ["Mechanical", "Electrical", "Hydraulic", "Pneumatic"]
locations = ["New York", "Texas", "California", "Illinois"]

# Create part info
part_info = []
for i in range(30):
    part_info.append({
        "PartID": f"P{str(i+1).zfill(3)}",
        "PartName": real_part_names[i],
        "Category": random.choice(categories),
        "Location": random.choice(locations),
        "Current_Stock": np.random.randint(50, 500),
        "LeadTime_Days": np.random.randint(3, 15),
        "Daily_Demand": np.random.randint(5, 25),
        "StdDev_Demand": round(np.random.uniform(1.0, 6.0), 2),
        "UnitCost": round(np.random.uniform(15.0, 150.0), 2)
    })

df_parts = pd.DataFrame(part_info)

# Generate 90-day demand history per part
start_date = datetime.today() - timedelta(days=90)
date_range = [start_date + timedelta(days=i) for i in range(90)]

demand_records = []
for part in part_info:
    for date in date_range:
        qty = max(0, int(np.random.normal(part["Daily_Demand"], part["StdDev_Demand"])))
        demand_records.append({
            "Date": date.date(),
            "PartID": part["PartID"],
            "PartName": part["PartName"],
            "Category": part["Category"],
            "Location": part["Location"],
            "Demand_Qty": qty
        })

df_demand = pd.DataFrame(demand_records)

# Save updated Excel files
parts_file = r"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Parts_Info.xlsx"
demand_file = r"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Daily_Demand_History.xlsx"

df_parts.to_excel(parts_file, index=False)
df_demand.to_excel(demand_file, index=False)

df_parts.head()

,PartID,PartName,Category,Location,Current_Stock,LeadTime_Days,Daily_Demand,StdDev_Demand,UnitCost
0,P001,Hydraulic Pump,Mechanical,New York,152,6,19,4.66,95.82
1,P002,Pressure Sensor,Hydraulic,Texas,152,12,23,1.50,77.00
2,P003,Check Valve,Electrical,Texas,422,6,12,4.54,17.78
3,P004,Ball Bearing,Mechanical,New York,307,10,16,5.69,15.11
4,P005,Seal Ring,Pneumatic,New York,493,7,5,2.52,85.84


### Add inventory logic

In [23]:
import pandas as pd

# Load the updated parts info file
df_parts = pd.read_excel(r"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Parts_Info.xlsx")

# Define Z-score for 95% service level
Z = 1.65

# Calculate inventory logic
df_parts["Safety_Stock"] = (Z * df_parts["StdDev_Demand"] * (df_parts["LeadTime_Days"] ** 0.5)).round(2)
df_parts["Reorder_Point"] = (df_parts["Daily_Demand"] * df_parts["LeadTime_Days"] + df_parts["Safety_Stock"]).round(2)
df_parts["Runout_Days"] = (df_parts["Current_Stock"] / df_parts["Daily_Demand"]).round(1)

# Classify stock status
def stock_status(row):
    if row["Current_Stock"] < row["Reorder_Point"]:
        return "Understocked"
    elif row["Current_Stock"] > (1.5 * row["Reorder_Point"]):
        return "Overstocked"
    else:
        return "OK"

df_parts["Stock_Status"] = df_parts.apply(stock_status, axis=1)

# Save to new Excel file
enhanced_file = R"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Analyzed_With_Logic.xlsx"
df_parts.to_excel(enhanced_file, index=False)

df_parts.head()

,PartID,PartName,Category,Location,Current_Stock,LeadTime_Days,Daily_Demand,StdDev_Demand,UnitCost,Safety_Stock,Reorder_Point,Runout_Days,Stock_Status
0,P001,Hydraulic Pump,Mechanical,New York,152,6,19,4.66,95.82,18.83,132.83,8.0,OK
1,P002,Pressure Sensor,Hydraulic,Texas,152,12,23,1.50,77.00,8.57,284.57,6.6,Understocked
2,P003,Check Valve,Electrical,Texas,422,6,12,4.54,17.78,18.35,90.35,35.2,Overstocked
3,P004,Ball Bearing,Mechanical,New York,307,10,16,5.69,15.11,29.69,189.69,19.2,Overstocked
4,P005,Seal Ring,Pneumatic,New York,493,7,5,2.52,85.84,11.00,46.00,98.6,Overstocked


### Finding the discrepancies

In [24]:
# Load the uploaded files
parts_file = R"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Analyzed_With_Logic.xlsx"
demand_file = R"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Daily_Demand_History.xlsx"

df_parts = pd.read_excel(parts_file)
df_demand = pd.read_excel(demand_file)

# Sample validation checks
sample_parts = df_parts.head()
sample_demand = df_demand.head()

# Check if Safety Stock, Reorder Point, and Stock Status make sense
df_parts["Expected_Reorder_Point"] = (
    df_parts["Daily_Demand"] * df_parts["LeadTime_Days"] +
    1.65 * df_parts["StdDev_Demand"] * np.sqrt(df_parts["LeadTime_Days"])
).round(2)

df_parts["Expected_Stock_Status"] = df_parts.apply(
    lambda row: "Understocked" if row["Current_Stock"] < row["Reorder_Point"]
    else "Overstocked" if row["Current_Stock"] > (1.5 * row["Reorder_Point"])
    else "OK", axis=1
)

# Compare expected and actual
discrepancies = df_parts[
    (df_parts["Reorder_Point"] != df_parts["Expected_Reorder_Point"]) |
    (df_parts["Stock_Status"] != df_parts["Expected_Stock_Status"])
]

discrepancies[[
    "PartID", "Reorder_Point", "Expected_Reorder_Point",
    "Stock_Status", "Expected_Stock_Status"
]]

,PartID,Reorder_Point,Expected_Reorder_Point,Stock_Status,Expected_Stock_Status


### Using statsmodels for Forecasting (Simple Exponential Smoothing)

In [26]:
import pandas as pd
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

# Load daily demand history
df_demand = pd.read_excel(R"C:\Users\Ashok Kasaram\Downloads\Updated_Inventory_Daily_Demand_History.xlsx")
df_demand["Date"] = pd.to_datetime(df_demand["Date"])

# Forecast horizon
forecast_days = 15

# Container for all forecasts
all_forecasts = []

# Forecast demand for each part
for part_id, group in df_demand.groupby("PartID"):
    group = group.sort_values("Date")
    part_name = group["PartName"].iloc[0]
    category = group["Category"].iloc[0]
    location = group["Location"].iloc[0]

    # Fit Simple Exponential Smoothing model
    try:
        model = SimpleExpSmoothing(group["Demand_Qty"]).fit(smoothing_level=0.2, optimized=False)
        forecast_values = model.forecast(forecast_days)

        forecast_dates = pd.date_range(start=group["Date"].max() + pd.Timedelta(days=1), periods=forecast_days)
        forecast_df = pd.DataFrame({
            "Date": forecast_dates,
            "Forecast_Demand": forecast_values.round(2),
            "PartID": part_id,
            "PartName": part_name,
            "Category": category,
            "Location": location
        })
        all_forecasts.append(forecast_df)
    except:
        continue  # Skip parts that error out (e.g., constant demand)

# Combine all forecasts
df_forecast_all = pd.concat(all_forecasts, ignore_index=True)

# Save to Excel
forecast_path = R"C:\Users\Ashok Kasaram\Downloads\Simple_Forecasted_Demand_Per_Part.xlsx"
df_forecast_all.to_excel(forecast_path, index=False)

df_forecast_all.head()

C:\Users\Ashok Kasaram\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\Ashok Kasaram\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\Ashok Kasaram\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\Ashok Kasaram\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To u

,Date,Forecast_Demand,PartID,PartName,Category,Location
0,2025-07-27,19.27,P001,Hydraulic Pump,Mechanical,New York
1,2025-07-28,19.27,P001,Hydraulic Pump,Mechanical,New York
2,2025-07-29,19.27,P001,Hydraulic Pump,Mechanical,New York
3,2025-07-30,19.27,P001,Hydraulic Pump,Mechanical,New York
4,2025-07-31,19.27,P001,Hydraulic Pump,Mechanical,New York
